In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [43]:
class FanVoteEstimator:
    def __init__(
        self,
        method="rank",
        lambda_constraint=1000.0,
        lambda_smooth=1.0,
        epsilon=1e-6,
        use_features=False,
        feature_names=None,
    ):
        self.method = method
        self.lambda_constraint = lambda_constraint
        self.lambda_smooth = lambda_smooth
        self.epsilon = epsilon
        self.use_features = use_features
        self.feature_names = feature_names

        # 将在fit中初始化
        self.N = None
        self.T = None
        self.params = None
        self.X = None
        self.e = None
        self.elimination_weeks = None
        self.F = None
        self.objective_history = []  # 添加优化历史记录
        self.violation_history = []  # 添加违反历史记录

    def fit(self, X, e, elimination_weeks=None, features=None, max_iter=5000, tol=1e-4):
        """
        features: 特征矩阵 [N x D]
        """
        self.X = X
        self.e = e
        self.objective_history = []
        self.violation_history = []

        if elimination_weeks is None:
            self.elimination_weeks = list(range(len(e)))
        else:
            self.elimination_weeks = elimination_weeks

        self.N, self.T = X.shape

        # 存储特征矩阵
        if features is not None and self.use_features:
            self.F = features
            D = features.shape[1]
        else:
            self.F = None
            D = 0

        # 参数向量结构
        if self.use_features and self.F is not None:
            total_params = D + 1 + self.T
        else:
            total_params = self.N + 1 + self.T

        # 改进的初始化策略
        initial_params = self._initialize_parameters(total_params, D if self.use_features else None)
        
        print(f"Parameter initialization range: [{initial_params.min():.4f}, {initial_params.max():.4f}]")

        # 使用SciPy优化，添加边界约束
        bounds = self._create_parameter_bounds(total_params, D if self.use_features else None)
        
        result = minimize(
            self.objective_function,
            initial_params,
            method="L-BFGS-B",
            bounds=bounds,
            options={"maxiter": max_iter, "ftol": tol, "gtol": tol, "disp": True, "maxls": 50},
            callback=self._optimization_callback
        )

        self.params = result.x
        self.fan_votes_ = self._compute_fan_votes(self.params)
        
        print(f"Optimization completed. Success: {result.success}")
        print(f"Final objective value: {result.fun:.6f}")
        print(f"Number of iterations: {result.nit}")
        print(f"Termination message: {result.message}")

        return self

    def _initialize_parameters(self, total_params, D=None):
        """改进的参数初始化策略"""
        params = np.zeros(total_params)
        
        if self.use_features and D is not None:
            # 使用特征：参数顺序为 [w (D维), β (1维), γ (T维)]
            params[:D] = np.random.randn(D) * 0.1  # 特征权重较小
            params[D] = 0.5 + np.random.randn() * 0.1  # β为正数，评委分数应有正影响
            params[D+1:] = np.random.randn(self.T) * 0.2  # 时间效应
        else:
            # 不使用特征：参数顺序为 [α (N维), β (1维), γ (T维)]
            params[:self.N] = np.random.randn(self.N) * 0.5  # 选手效应，变化较大
            params[self.N] = 0.5 + np.random.randn() * 0.1  # β为正数
            params[self.N+1:] = np.random.randn(self.T) * 0.2  # 时间效应
            
        return params

    def _create_parameter_bounds(self, total_params, D=None):
        """为参数创建边界约束"""
        bounds = []
        
        if self.use_features and D is not None:
            # w: 特征权重，无约束（但实际可能有边界）
            for _ in range(D):
                bounds.append((None, None))
            # β: 评委影响系数，应为正数
            bounds.append((0.1, 10.0))
            # γ: 时间效应
            for _ in range(self.T):
                bounds.append((-5.0, 5.0))
        else:
            # α: 选手效应
            for _ in range(self.N):
                bounds.append((-5.0, 5.0))
            # β: 评委影响系数，应为正数
            bounds.append((0.1, 10.0))
            # γ: 时间效应
            for _ in range(self.T):
                bounds.append((-5.0, 5.0))
                
        return bounds

    def _optimization_callback(self, xk):
        """优化回调函数，记录优化过程"""
        current_obj = self.objective_function(xk, track_history=True)
        V_current = self._compute_fan_votes(xk)
        
        # 计算总约束违反
        total_violation = 0.0
        for week in self.elimination_weeks:
            violation = self._constraint_violation(week, V_current)
            if violation > 0:
                total_violation += violation
                
        self.violation_history.append(total_violation)
        
        # 每50次迭代打印一次
        if len(self.objective_history) % 50 == 0:
            print(f"Iteration {len(self.objective_history)}: obj={current_obj:.4f}, total_violation={total_violation:.4f}")
            print(f"  V range: [{V_current.min():.2e}, {V_current.max():.2e}]")
            
        return False

    def _compute_fan_votes(self, params, debug=False):
        """从参数计算粉丝投票矩阵，添加数值稳定性"""
        N, T = self.N, self.T

        if self.use_features and self.F is not None:
            # 使用特征线性组合计算 α_i = w^T * F_i
            D = self.F.shape[1]
            w = params[:D]
            beta = params[D]
            gamma = params[D + 1:]

            # 计算 α_i 向量
            alpha = self.F @ w  # [N x 1]
        else:
            # 原来的独立参数
            alpha = params[:N]
            beta = params[N]
            gamma = params[N + 1:]

        # 计算 log(V) = α_i + βX + γ_t
        log_V = alpha[:, np.newaxis] + beta * self.X + gamma[np.newaxis, :]
        
        # 数值稳定性：防止溢出/下溢
        log_V = np.clip(log_V, -30, 30)  # 扩大clip范围
        V = np.exp(log_V)
        
        # 防止完全为0，添加小常数
        V = V + 1e-10
        
        if debug:
            print(f"Debug _compute_fan_votes:")
            print(f"  alpha range: [{alpha.min():.4f}, {alpha.max():.4f}]")
            print(f"  beta: {beta:.4f}")
            print(f"  gamma range: [{gamma.min():.4f}, {gamma.max():.4f}]")
            print(f"  log_V range: [{log_V.min():.4f}, {log_V.max():.4f}]")
            print(f"  V range: [{V.min():.2e}, {V.max():.2e}]")
            print(f"  V mean: {V.mean():.2e}, std: {V.std():.2e}")
            # 检查是否有异常值
            zero_count = np.sum(V < 1e-5)
            if zero_count > 0:
                print(f"  WARNING: {zero_count} values close to zero (< 1e-5)")
        
        return V

    def _active_contestants(self, week):
        """返回第week周仍在比赛的选手索引列表"""
        if week == 0:
            return list(range(self.N))
        else:
            # 找出在week周之前被淘汰的选手
            eliminated_before = []
            for i, elim_week in enumerate(self.elimination_weeks):
                if elim_week < week:
                    eliminated_before.append(self.e[i])
            return [i for i in range(self.N) if i not in eliminated_before]

    def _constraint_violation(self, week, V):
        """计算第week周的约束违反量，改进版本"""
        # 如果这周没有淘汰，返回0
        if week not in self.elimination_weeks:
            return 0.0
        
        # 找出这周的淘汰者
        elim_idx = self.elimination_weeks.index(week)
        e_t = self.e[elim_idx]
        
        active = self._active_contestants(week)
        if e_t not in active:
            return 0.0
        
        e_idx_in_active = active.index(e_t)
        week_X = self.X[active, week]
        week_V = V[active, week]
        
        # 检查数据有效性
        if week_X.sum() == 0 or week_V.sum() == 0:
            return 0.0  # 数据无效，跳过
        
        if self.method == 'rank':
            # 计算排名（降序：分数高排名小）
            X_rank = self._compute_rank(-week_X)  # 负号因为分数高排名好
            V_rank = self._compute_rank(-week_V)
            combined = X_rank + V_rank
            
            # 淘汰者应有最大综合排名（即排名数字最大）
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                max_others = np.max(combined_others)
            else:
                max_others = combined[e_idx_in_active]
            
            # violation = max_others - combined[e_idx_in_active] + epsilon
            # 如果淘汰者的综合排名不是最大，则违反约束
            violation = max(0.0, max_others - combined[e_idx_in_active] + self.epsilon)
        
        else:  # 'percent'
            # 计算百分比
            X_percent = week_X / week_X.sum()
            V_percent = week_V / week_V.sum()
            combined = X_percent + V_percent
            
            # 淘汰者应有最小综合百分比
            combined_others = np.delete(combined, e_idx_in_active)
            if len(combined_others) > 0:
                min_others = np.min(combined_others)
            else:
                min_others = combined[e_idx_in_active]
            
            violation = max(0.0, combined[e_idx_in_active] - min_others + self.epsilon)
        
        return violation

    def _compute_rank(self, scores):
        """计算排名，处理并列情况"""
        # 使用稳定的排名方法
        order = np.argsort(scores)
        ranks = np.empty_like(order)
        ranks[order] = np.arange(len(scores))
        
        # 处理并列：相同分数得到相同排名
        unique_scores, inverse, counts = np.unique(
            scores, return_inverse=True, return_counts=True
        )
        
        for i in range(len(unique_scores)):
            if counts[i] > 1:
                indices = np.where(inverse == i)[0]
                # 使用平均排名
                avg_rank = np.mean(ranks[indices])
                ranks[indices] = avg_rank
        
        return ranks + 1  # 排名从1开始

    def objective_function(self, params, track_history=False):
        """改进的目标函数，添加正则化"""
        V = self._compute_fan_votes(params)
        
        # 1. 约束惩罚项
        constraint_penalty = 0.0
        weekly_violations = []
        
        for week in self.elimination_weeks:
            violation = self._constraint_violation(week, V)
            weekly_violations.append(violation)
            if violation > 0:
                constraint_penalty += violation**2
        
        # 2. 平滑项
        smooth_penalty = np.sum((V[:, 1:] - V[:, :-1]) ** 2)
        
        # 3. 参数正则化（防止过拟合）
        param_penalty = np.sum(params**2) * 0.01
        
        # 4. 总目标
        total = (
            self.lambda_constraint * constraint_penalty
            + self.lambda_smooth * smooth_penalty
            + param_penalty
        )
        
        if track_history:
            self.objective_history.append(total)
            
        return total

    def predict(self, X=None):
        """返回估计的粉丝投票"""
        if X is None:
            X = self.X
        return self.fan_votes_
    
    def set_voting_method(self, method):
        """设置投票方法: 'rank' 或 'percent'"""
        if method not in ['rank', 'percent']:
            raise ValueError("method must be 'rank' or 'percent'")
        self.method = method
    
    def get_optimization_history(self):
        """返回优化历史"""
        return {
            'objective': self.objective_history,
            'violation': self.violation_history
        }

In [44]:
def prepare_season_data_with_features(season_num, raw_data, use_features=True):
    """
    为指定赛季准备数据，包括特征
    返回: X, e, names, elimination_weeks, features
    """
    # 筛选指定赛季的数据
    season_data = raw_data[raw_data["season"] == season_num].copy()

    # 重置索引
    season_data = season_data.reset_index(drop=True)

    # 提取选手名称
    contestant_names = season_data["celebrity_name"].tolist()
    N = len(contestant_names)

    # 确定最大周数
    max_week = 0
    for col in season_data.columns:
        if "week" in col and "judge" in col:
            week_num = int(col.split("_")[0].replace("week", ""))
            max_week = max(max_week, week_num)

    # 初始化评委分数矩阵
    X = np.zeros((N, max_week))

    # 填充评委分数矩阵
    for i in range(N):
        for week in range(1, max_week + 1):
            week_cols = [col for col in season_data.columns if f"week{week}_" in col]
            week_total = 0
            judge_count = 0

            for col in week_cols:
                score = season_data.at[i, col]
                if pd.isna(score) or score == "N/A" or score == 0:
                    continue
                try:
                    score_val = float(score)
                    week_total += score_val
                    judge_count += 1
                except:
                    continue

            if judge_count > 0:
                X[i, week - 1] = week_total / judge_count

    # 解析淘汰信息（同之前）
    # ... [保持原来的淘汰信息解析代码不变] ...

    # 特征工程
    features = None
    if use_features:
        features = extract_features(season_data)

    # 确定实际需要处理的周数
    if len(elimination_weeks) > 0:
        T = max(elimination_weeks) + 1
    else:
        T = max_week

    # 截断X矩阵到T周
    X = X[:, :T]

    return X, e, contestant_names, elimination_weeks, features

def extract_features(data):
    """
    从原始数据中提取特征
    返回特征矩阵 [N x D]
    """
    N = len(data)
    features_list = []

    for i in range(N):
        feature_vector = []

        # 1. 年龄特征
        age = data.at[i, "celebrity_age_during_season"]
        if pd.isna(age):
            age = 40  # 默认值
        feature_vector.append(float(age))

        # 2. 行业特征（独热编码）
        industry = str(data.at[i, "celebrity_industry"])
        industries = [
            "Actor/Actress",
            "Athlete",
            "Singer/Rapper",
            "Model",
            "TV Personality",
            "News Anchor",
            "Politician",
            "Other",
        ]

        industry_encoded = [0] * len(industries)
        if industry in industries:
            industry_encoded[industries.index(industry)] = 1
        else:
            industry_encoded[-1] = 1  # 'Other'类别
        feature_vector.extend(industry_encoded)

        # 3. 地区特征（简化：美国/非美国）
        country = str(data.at[i, "celebrity_homecountry/region"])
        is_usa = 1 if "United States" in country else 0
        feature_vector.append(is_usa)

        # 4. 是否为体育明星（额外特征）
        is_athlete = 1 if "Athlete" in industry else 0
        feature_vector.append(is_athlete)

        features_list.append(feature_vector)

    features = np.array(features_list)

    # 对数值特征进行标准化
    numeric_cols = [0]  # 年龄列
    for col in numeric_cols:
        mean_val = np.mean(features[:, col])
        std_val = np.std(features[:, col])
        if std_val > 0:
            features[:, col] = (features[:, col] - mean_val) / std_val

    return features

In [45]:
# 修改prepare_season_data函数，添加更多调试信息
def prepare_season_data(season_num, raw_data, debug=False):
    """
    为指定赛季准备数据
    返回: X(评委分数矩阵), e(淘汰顺序), contestant_names(选手名称), elimination_weeks(淘汰周列表)
    """
    # 筛选指定赛季的数据
    season_data = raw_data[raw_data["season"] == season_num].copy()

    if debug:
        print(f"Season {season_num}: {len(season_data)} contestants")

    # 重置索引
    season_data = season_data.reset_index(drop=True)

    # 提取选手名称
    contestant_names = season_data["celebrity_name"].tolist()
    N = len(contestant_names)

    # 确定最大周数（从列名中提取）
    max_week = 0
    for col in season_data.columns:
        if "week" in col and "judge" in col:
            try:
                week_num = int(col.split("_")[0].replace("week", ""))
                max_week = max(max_week, week_num)
            except:
                continue

    if debug:
        print(f"  Max weeks from columns: {max_week}")

    # 初始化评委分数矩阵
    X = np.zeros((N, max_week))

    # 填充评委分数矩阵
    for i in range(N):
        for week in range(1, max_week + 1):
            # 获取本周所有评委分数列
            week_cols = [col for col in season_data.columns if f"week{week}_" in col]
            week_total = 0
            judge_count = 0

            for col in week_cols:
                score = season_data.at[i, col]
                # 处理N/A和0值
                if pd.isna(score) or score == "N/A" or score == 0:
                    continue
                try:
                    score_val = float(score)
                    week_total += score_val
                    judge_count += 1
                except:
                    continue

            # 如果本周有评委分数，则记录平均值
            if judge_count > 0:
                X[i, week - 1] = week_total / judge_count
            else:
                # 如果没有评委分数，使用前一周的分数（如果有）
                if week > 1 and X[i, week - 2] > 0:
                    X[i, week - 1] = X[i, week - 2]

    if debug:
        print(f"  X matrix: {X.shape}")
        print(f"  X non-zero values: {np.sum(X > 0)} / {X.size}")

    # 解析淘汰信息
    elimination_info = []

    for i in range(N):
        result = str(season_data.at[i, "results"])

        # 检查是否是决赛选手
        if any(
            place in result
            for place in ["1st", "2nd", "3rd", "4th", "5th", "Winner", "Champion"]
        ):
            # 决赛选手：在最后一周被"淘汰"（实际上不被淘汰，但为了模型需要）
            elim_week = max_week - 1  # 0-based索引
            if debug:
                print(f"  {contestant_names[i]}: Finalist, elim_week = {elim_week+1}")
        elif "Withdrew" in result or "withdrew" in result:
            # 退赛情况
            elim_week = max_week - 1  # 默认设为最后一周
            for week in range(max_week):
                if X[i, week] == 0 and week > 0:
                    elim_week = week - 1
                    break
            if debug:
                print(f"  {contestant_names[i]}: Withdrew, elim_week = {elim_week+1}")
        elif "Eliminated Week" in result:
            # 解析淘汰周次
            try:
                week_str = result.replace("Eliminated Week", "").strip()
                elim_week = int(week_str) - 1  # 转换为0-based索引
                if debug:
                    print(
                        f"  {contestant_names[i]}: Eliminated Week {week_str}, elim_week = {elim_week+1}"
                    )
            except:
                elim_week = max_week - 1
        elif "Eliminated" in result and "Week" in result:
            # 其他格式的淘汰信息
            try:
                import re

                week_match = re.search(r"Week\s*(\d+)", result)
                if week_match:
                    elim_week = int(week_match.group(1)) - 1
                else:
                    elim_week = max_week - 1
            except:
                elim_week = max_week - 1
        else:
            # 其他情况，设为最后一周
            elim_week = max_week - 1
            if debug and "Place" not in result:
                print(
                    f"  {contestant_names[i]}: Unknown result '{result}', set elim_week = {elim_week+1}"
                )

        elimination_info.append(
            {
                "index": i,
                "elim_week": elim_week,
                "name": contestant_names[i],
                "result": result,
            }
        )

    # 按淘汰周次排序
    def sort_key(info):
        week = info["elim_week"]
        score = X[info["index"], week] if week < max_week else 0
        return (week, score)

    elimination_info.sort(key=sort_key)

    # 构建淘汰顺序数组e和淘汰周列表
    e = []
    elimination_weeks = []

    for info in elimination_info:
        # 跳过冠军（最后一周不被淘汰）
        if (
            "1st" in info["result"]
            or "Winner" in info["result"]
            or "Champion" in info["result"]
        ):
            continue

        e.append(info["index"])
        elimination_weeks.append(info["elim_week"])

    # 转换为numpy数组
    e = np.array(e)
    elimination_weeks = np.array(elimination_weeks)

    # 确定实际需要处理的周数（最大的淘汰周次+1）
    if len(elimination_weeks) > 0:
        T = max(elimination_weeks) + 1
    else:
        T = max_week

    # 截断X矩阵到T周
    X = X[:, :T]

    # 标准化评委分数（可选）
    # 为了让不同赛季尺度一致，可以标准化评委分数
    X_mean = X[X > 0].mean() if np.any(X > 0) else 1.0
    X_std = X[X > 0].std() if np.any(X > 0) else 1.0
    if X_std > 0:
        X_normalized = (X - X_mean) / X_std
    else:
        X_normalized = X.copy()

    if debug:
        print(f"  Final: {N} contestants, {T} weeks")
        print(f"  Elimination weeks: {elimination_weeks + 1}")
        print(f"  Elimination order: {[contestant_names[idx] for idx in e]}")
        print(
            f"  X normalized range: [{X_normalized.min():.2f}, {X_normalized.max():.2f}]"
        )

    return X_normalized, e, contestant_names, elimination_weeks


# 改进的analyze_season函数
def analyze_season(
    season_num, method="rank", lambda_constraint=1000.0, lambda_smooth=1.0, debug=False
):
    """分析单个赛季"""
    print(f"\n{'='*60}")
    print(f"Analyzing Season {season_num} using {method} method")
    print("=" * 60)

    try:
        # 准备数据
        X, e, names, elimination_weeks = prepare_season_data(
            season_num, raw_data, debug=debug
        )

        if len(e) == 0:
            print(f"Season {season_num} has no elimination data. Skipping...")
            return None

        print(f"X shape: {X.shape}, e length: {len(e)}")
        print(f"Elimination weeks (1-based): {elimination_weeks + 1}")

        # 创建估计器
        estimator = FanVoteEstimator(
            method=method,
            lambda_constraint=lambda_constraint,
            lambda_smooth=lambda_smooth,
        )

        # 拟合模型
        print("\nFitting model...")
        estimator.fit(X, e, elimination_weeks=elimination_weeks.tolist())

        # 获取估计的粉丝投票
        V_estimated = estimator.fan_votes_

        # 检查投票值的数值稳定性
        print(f"\nFan votes statistics:")
        print(f"  Min: {V_estimated.min():.6e}")
        print(f"  Max: {V_estimated.max():.6e}")
        print(f"  Mean: {V_estimated.mean():.6e}")
        print(f"  Std: {V_estimated.std():.6e}")

        zero_count = np.sum(V_estimated < 1e-5)
        if zero_count > 0:
            print(f"  WARNING: {zero_count} values close to zero (< 1e-5)")

        # 检查约束满足情况
        print("\nConstraint violations:")
        violations = []
        constraint_details = []

        for week_idx, week in enumerate(elimination_weeks):
            violation = estimator._constraint_violation(week, V_estimated)
            violations.append(violation)
            actual_week = week + 1

            # 获取详细信息
            active = estimator._active_contestants(week)
            if week in elimination_weeks:
                elim_idx = elimination_weeks.tolist().index(week)
                e_t = e[elim_idx]
                if e_t in active:
                    e_idx_in_active = active.index(e_t)
                    week_X = X[active, week]
                    week_V = V_estimated[active, week]

                    if method == "rank":
                        X_rank = estimator._compute_rank(-week_X)
                        V_rank = estimator._compute_rank(-week_V)
                        combined = X_rank + V_rank
                        constraint_details.append(
                            {
                                "week": actual_week,
                                "eliminated": names[e_t],
                                "X_rank": X_rank[e_idx_in_active],
                                "V_rank": V_rank[e_idx_in_active],
                                "combined_rank": combined[e_idx_in_active],
                                "violation": violation,
                            }
                        )

            status = "✓" if violation <= estimator.epsilon else "✗"
            print(f"Week {actual_week:2d}: {status} violation = {violation:.6f}")

        if violations:
            avg_violation = np.mean(violations)
            max_violation = np.max(violations)
            print(f"\nAverage violation: {avg_violation:.6f}")
            print(f"Maximum violation: {max_violation:.6f}")

            # 计算约束满足率
            satisfied = sum(1 for v in violations if v <= estimator.epsilon)
            satisfaction_rate = satisfied / len(violations) * 100
            print(
                f"Constraints satisfied: {satisfied}/{len(violations)} ({satisfaction_rate:.1f}%)"
            )

        # 显示估计结果
        print(f"\nEstimated fan votes (first week, normalized):")
        first_week_votes = V_estimated[:, 0]
        # 归一化显示
        if first_week_votes.sum() > 0:
            first_week_percent = first_week_votes / first_week_votes.sum() * 100
            sorted_indices = np.argsort(-first_week_percent)
            for i, idx in enumerate(sorted_indices[:5]):
                print(
                    f"  {i+1}. {names[idx]:20s}: {first_week_percent[idx]:5.1f}% (raw: {first_week_votes[idx]:.4f})"
                )

        # 显示最后一周的投票
        if V_estimated.shape[1] > 1:
            last_week = V_estimated.shape[1] - 1
            last_week_votes = V_estimated[:, last_week]
            if last_week_votes.sum() > 0:
                last_week_percent = last_week_votes / last_week_votes.sum() * 100
                print(f"\nEstimated fan votes (last week, normalized):")
                sorted_indices = np.argsort(-last_week_percent)
                for i, idx in enumerate(sorted_indices[:5]):
                    print(
                        f"  {i+1}. {names[idx]:20s}: {last_week_percent[idx]:5.1f}% (raw: {last_week_votes[idx]:.4f})"
                    )

        # 返回结果
        result = {
            "season": season_num,
            "X": X,
            "e": e,
            "names": names,
            "V": V_estimated,
            "violations": violations,
            "elimination_weeks": elimination_weeks,
            "avg_violation": avg_violation if violations else 0,
            "max_violation": max_violation if violations else 0,
            "satisfaction_rate": satisfaction_rate if violations else 0,
            "estimator": estimator,
        }

        return result

    except Exception as error:
        print(f"Error analyzing season {season_num}: {error}")
        import traceback

        traceback.print_exc()
        return None


def analyze_multiple_seasons(
    seasons, method="rank", lambda_constraint=1000.0, lambda_smooth=1.0
):
    """分析多个赛季"""
    all_results = []

    for season in seasons:
        result = analyze_season(season, method, lambda_constraint, lambda_smooth)
        if result:
            all_results.append(result)

    return all_results


def summarize_results(results):
    """汇总分析结果"""
    if not results:
        print("No results to summarize.")
        return

    print(f"\n{'='*80}")
    print("SUMMARY ACROSS SEASONS")
    print("=" * 80)
    print(
        f"{'Season':<8} {'Contestants':<12} {'Weeks':<8} {'Avg Violation':<15} {'Max Violation':<15} {'Satisfaction':<12}"
    )
    print("-" * 80)

    for res in results:
        season = res["season"]
        contestants = len(res["names"])
        weeks = res["X"].shape[1]
        avg_viol = res["avg_violation"]
        max_viol = res["max_violation"]
        satisfaction = res["satisfaction_rate"]

        print(
            f"{season:<8} {contestants:<12} {weeks:<8} {avg_viol:<15.6f} {max_viol:<15.6f} {satisfaction:<12.1f}%"
        )

    # 计算总体统计
    if results:
        print("-" * 80)
        total_seasons = len(results)
        avg_avg_viol = np.mean([r["avg_violation"] for r in results])
        avg_max_viol = np.mean([r["max_violation"] for r in results])
        avg_satisfaction = np.mean([r["satisfaction_rate"] for r in results])

        print(
            f"{'AVERAGE':<8} {'-':<12} {'-':<8} {avg_avg_viol:<15.6f} {avg_max_viol:<15.6f} {avg_satisfaction:<12.1f}%"
        )
        print(f"\nTotal seasons analyzed: {total_seasons}")

In [47]:
def main():
    """主执行函数"""
    # 加载数据
    global raw_data
    data_path = "2026_MCM_Problem_C_Data.csv"
    raw_data = pd.read_csv(data_path)

    # 显示数据基本信息
    print("=" * 80)
    print("DANCING WITH THE STARS - FAN VOTE ESTIMATION MODEL")
    print("=" * 80)
    print(f"\nData shape: {raw_data.shape}")
    print(f"Seasons available: {sorted(raw_data['season'].unique())}")
    print(f"Total contestants: {len(raw_data)}")

    # 分析所有赛季（或选择特定赛季）
    all_seasons = sorted(raw_data["season"].unique())

    # 为了演示，只分析前10个赛季
    # seasons_to_analyze = all_seasons[:10]
    seasons_to_analyze = all_seasons
    print(f"\nAnalyzing seasons: {seasons_to_analyze}")

    # 设置模型参数
    method = "rank"  # 或 "percent"
    lambda_constraint = 1000.0
    lambda_smooth = 1.0

    # 分析多个赛季
    results = analyze_multiple_seasons(
        seasons=seasons_to_analyze,
        method=method,
        lambda_constraint=lambda_constraint,
        lambda_smooth=lambda_smooth,
    )

    # 汇总结果
    summarize_results(results)

    # 可选：详细分析特定有争议的赛季
    print(f"\n\n{'='*80}")
    print("SPECIAL ANALYSIS: CONTROVERSIAL SEASONS")
    print("=" * 80)

    controversial_seasons = [2, 4, 11, 27]  # 根据问题描述

    for season in controversial_seasons:
        if season in all_seasons:
            print(f"\nAnalyzing controversial Season {season}:")
            result = analyze_season(season, method, lambda_constraint, lambda_smooth)
            if result:
                # 找出有争议的选手                print(f"\nControversial contestants in Season {season}:")
                for i, name in enumerate(result["names"]):
                    result_str = raw_data[
                        (raw_data["season"] == season)
                        & (raw_data["celebrity_name"] == name)
                    ]["results"].iloc[0]
                    if any(
                        term in str(result_str)
                        for term in ["controvers", "low score", "despite"]
                    ):
                        print(f"  {name}: {result_str}")

    print("\n" + "=" * 80)
    print("ANALYSIS COMPLETE")
    print("=" * 80)


if __name__ == "__main__":
    main()

DANCING WITH THE STARS - FAN VOTE ESTIMATION MODEL

Data shape: (421, 53)
Seasons available: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29), np.int64(30), np.int64(31), np.int64(32), np.int64(33), np.int64(34)]
Total contestants: 421

Analyzing seasons: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19), np.int64(20), np.int64(21), np.int64(22), np.int64(23), np.int64(24), np.int64(25), np.int64(26), np.int64(27), np.int64(28), np.int64(29)

C:\Users\pl050\AppData\Local\Temp\ipykernel_28312\3889974929.py:67: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  result = minimize(


Optimization completed. Success: True
Final objective value: 63045.962645
Number of iterations: 3
Termination message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH

Fan votes statistics:
  Min: 8.093162e-02
  Max: 1.960771e+00
  Mean: 4.363550e-01
  Std: 4.271561e-01

Constraint violations:
Week  1: ✗ violation = 4.000001
Week  2: ✗ violation = 3.000001
Week  3: ✗ violation = 5.000001
Week  4: ✓ violation = 0.000001
Week  5: ✓ violation = 0.000001
Week  6: ✗ violation = 2.000001
Week  7: ✗ violation = 3.000001
Week 11: ✓ violation = 0.000000
Week 11: ✓ violation = 0.000000

Average violation: 1.888890
Maximum violation: 5.000001
Constraints satisfied: 4/9 (44.4%)

Estimated fan votes (first week, normalized):
  1. Tatum O'Neal        :  16.6% (raw: 1.9608)
  2. Kenny Mayne         :  14.6% (raw: 1.7196)
  3. Master P            :  12.6% (raw: 1.4851)
  4. Giselle Fernandez   :  10.6% (raw: 1.2546)
  5. Stacy Keibler       :   8.8% (raw: 1.0402)

Estimated fan votes (last week, 